In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sqlite3
from sqlalchemy import create_engine

import datetime

import pywt

In [34]:
ccy='BTCUSDT'
data_frequency='1h'
start_date='2022-01-01'
end_date='2022-12-31'

In [35]:
def get_and_process_data(ccy, data_frequency, start_date, end_date):
    try:
        
        # Get the data
        db_address = 'sqlite:///../../data/db/crypto.db'
        engine = create_engine(db_address, echo=True)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_all_features" + "_" + ccy + "_" + data_frequency
        
        sql_command = "SELECT * FROM " + input_tbl_name

        if(start_date!="" and end_date!=""):
            
            sd = datetime.datetime.strptime(start_date,'%Y-%m-%d').strftime('%Y-%m-%d')
            ed = datetime.datetime.strptime(end_date,'%Y-%m-%d').strftime('%Y-%m-%d')

            sql_append = " WHERE " + "date(date) >= " + "'" + sd + "'" + " AND date(date) <= " + "'" + ed + "'"
            sql_command = sql_command + sql_append

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        # Converting Date Column to DateTime Type
        # Set the index on the dataframe
        df['date_index'] = df['date']
        df.set_index('date_index', inplace=True)

        return df

    except Exception as e:
        print(e)

In [36]:
df = get_and_process_data(ccy, data_frequency, start_date, end_date)
x = df.index
y = df['close']

2023-01-31 17:13:48,486 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM tbl_all_features_BTCUSDT_1h WHERE date(date) >= '2022-01-01' AND date(date) <= '2022-12-31'")
2023-01-31 17:13:48,487 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-31 17:13:48,492 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM tbl_all_features_BTCUSDT_1h WHERE date(date) >= '2022-01-01' AND date(date) <= '2022-12-31'")
2023-01-31 17:13:48,492 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-31 17:13:48,493 INFO sqlalchemy.engine.Engine SELECT * FROM tbl_all_features_BTCUSDT_1h WHERE date(date) >= '2022-01-01' AND date(date) <= '2022-12-31'
2023-01-31 17:13:48,493 INFO sqlalchemy.engine.Engine [raw sql] ()


In [37]:
# perform wavelet decomposition
coeffs = pywt.wavedec(y, 'sym5', mode='symmetric')

In [38]:
# reconstruct wavelet
y_rec = pywt.waverec(coeffs, 'sym5', mode='symmetric')[1:]

In [39]:
# # plot result
# plt.figure(figsize=(18,6))
# plt.plot(x, y)
# plt.plot(x, y_rec)
# plt.legend(['original', 'wavelet-based smoothing'])
# plt.xlabel('time index')
# plt.show()

In [40]:
fig = go.Figure()

fig.add_trace(go.Scatter(name='close', x=x, y=y, mode='lines'))
fig.add_trace(go.Scatter(name='wavelet', x=x, y=y_rec, mode='lines'))

fig.update_layout(title="", width=1200, height=800)

fig.show()